In [1]:
from openff.toolkit.topology import Molecule

# QC Submit packages
from openff.qcsubmit.common_structures import QCSpec
from openff.qcsubmit.factories import TorsiondriveDatasetFactory
from openff.qcsubmit.workflow_components import (
    Scan1D, ScanEnumerator, StandardConformerGenerator
)
# Compute the training set, running calculations
from qcfractal import FractalSnowflakeHandler
from qcportal.client import FractalClient

In [10]:
# first build your moecules that you want to run torsion scans on
training_smiles= ['CC', 'CCCC']

training_molecules = [Molecule.from_smiles(smile) for smile in training_smiles]

## we now have 3 molecules, how do we turn these into a set of QC calucaltions?

In [11]:
torsion_drive_factory = TorsiondriveDatasetFactory(
    qc_specifications={
        "default": QCSpec(
            method="B3LYP-D3BJ", basis="DZVP", program="psi4"
        )
    },
    workflow=[
        ScanEnumerator(
            torsion_scans=[
                Scan1D(
                    smarts1="[*:1]-[#6:2]-[#6:3]-[*:4]",
                    scan_range1=(-150, 180),
                    scan_increment=[30]
                )
            ]
        ),
        StandardConformerGenerator(toolkit='rdkit',max_conformers=10)
    ],
)

## Create a Torsiondrive Dataset Factor that will produce a dataset of QC torsion drives

# supply: 
#       1. a set of QC Specifications (define the level of theory as well as the program with which we want to run out calcs.
#       we have used the openFF defaults which were carefully chosen to provide a balance of speed and accuracy at reproducing conformational energies)
#       2. a workflow (set of processing steps that will be applied to a set of moleucles. Gnerate a set of conformers to start torsion drives from, 
#       enumerating all tautomers and protomers, fragmenting input molecules to speed up QC calcs)


# conformer generator to make sure we have several conformers to start the torsion drive from 
# smarts pattern to define exactly what torsions should be scanned. Match all unique torsions around a C-C single bond

In [12]:
torsion_drive_dataset = torsion_drive_factory.create_dataset(
    molecules=training_molecules,
    dataset_name="Alkane Torsion Drives",
    description="A very basic dataset containing three linear alkanes: "
    "ethane, propane, and butane.",
    tagline="A very basic dataset containing three linear alkanes.",
)

Preparation                   : 100%|█████████████| 2/2 [00:00<00:00, 37.05it/s]


In [13]:
local_fractal_instance = FractalSnowflakeHandler(ncores=16)
local_fractal_client = FractalClient(local_fractal_instance)

ConnectionRefusedError: [Errno Snowflake instance did not boot properly, try increasing the timeout.

stdout:
Starting a QCFractal server.

QCFractal server base folder: /tmp/tmpa5r7697k

>>> Examining SSL Certificates...
Autogenerated SSL certificates, clients must use 'verify=False' when connecting.

>>> Logging to /tmp/tmpa5r7697k/db_80bf03ef_8d95_440c_9cec_ed6c0ed40a3e
>>> Loglevel: INFO

>>> Checking the PostgreSQL connection...

>>> Initializing the QCFractal server...
Fatal during server startup:

[SSL: EE_KEY_TOO_SMALL] ee key too small (_ssl.c:4048)

Failed to start the server, shutting down.


] stderr:


In [ ]:
submission = torsion_drive_dataset.submit(local_fractal_client)